### ДЗ № 11
Тема “Перевод с механизмом внимания”

Разобраться с моделькой перевода (с механизмом внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)

Модель перевода с вниманием была создана еще в ДЗ №10 

https://github.com/JuliPolu/GB_NLP_intro/blob/main/NLP_hw_les_10_translate.ipynb

Поэтому в этой ДЗ попробую протестировать трансформенную архитектуру

In [2]:
# !pip install transformers

In [1]:
import pandas as pd
import re
import io

import transformers

In [2]:
path_to_file = "../Less_10_Seq_to_Seq/rus-eng/rus.txt"

In [3]:
# Функция препроцессинг текста

def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [4]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [5]:
en, ru = create_dataset(path_to_file, None)
print(en[100])
print(ru[100])

<start> cheers ! <end>
<start> будем ! <end>


In [6]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

/Users/aleksandrkrylov/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [7]:
for i in range(10):
    input = ru[i]
    input_ids = tokenizer.encode(input, return_tensors="pt")
    output = model.generate(input_ids)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}, real: {en[i]}')

/Users/aleksandrkrylov/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ru: <start> марш ! <end>, en: <start> march! <end>, real: <start> go . <end>


/Users/aleksandrkrylov/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ru: <start> иди . <end>, en: <start> go. <end>, real: <start> go . <end>
ru: <start> идите . <end>, en: <start> go. <end>, real: <start> go . <end>
ru: <start> здравствуйте . <end>, en: <start> hello. <end>, real: <start> hi . <end>
ru: <start> привет ! <end>, en: <start> hello! <end>, real: <start> hi . <end>
ru: <start> хай . <end>, en: <start> hey. <end>, real: <start> hi . <end>
ru: <start> здрасте . <end>, en: <start> hello. <end>, real: <start> hi . <end>
ru: <start> здоро во ! <end>, en: <start> hello at! <end>, real: <start> hi . <end>
ru: <start> приветик ! <end>, en: <start> hello! <end>, real: <start> hi . <end>
ru: <start> беги ! <end>, en: #start> run! #end>, real: <start> run ! <end>


In [9]:
#Протеструем фразы из предыдущего ДЗ и добавим еще длинных фраз
inputs = ['Без труда не выловишь и рыбку из пруда', 'Сделал дело - гуляй смело', 
            'Не откладывай на завтра то, что можешь сделать сегодня', 'мама мыла окно',  
            'Маленькой ели холодно зимой', 'мы идем гулять по лесу', 'как мне пройти в библиотеку?',
            'Мне кажется, что все идет так, как должно идти',
            'Очень часто высказывания человека зависят от его субъективного мироощущения и настроения в данный момент времени.']
for input in inputs:
    input_ids = tokenizer.encode(input, return_tensors="pt")
    output = model.generate(input_ids)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}')

ru: Без труда не выловишь и рыбку из пруда, en: You can't catch a fish out of a pond without a hard time.
ru: Сделал дело - гуляй смело, en: You've done your job, you've got to go for a walk.
ru: Не откладывай на завтра то, что можешь сделать сегодня, en: Don't put off for tomorrow what you can do today.
ru: мама мыла окно, en: Mom was washing the window.
ru: Маленькой ели холодно зимой, en: The little one ate cold in the winter.
ru: мы идем гулять по лесу, en: We're going for a walk in the woods.
ru: как мне пройти в библиотеку?, en: How am I supposed to get to the library?
ru: Мне кажется, что все идет так, как должно идти, en: I think it's going the way it's supposed to go.
ru: Очень часто высказывания человека зависят от его субъективного мироощущения и настроения в данный момент времени., en: Very often, a person's speech depends on his subjective state of mind and mood at this point in time.


**Очевидно, что предобученная модель знаяительно лучше производит перевод**